# Imports playground

This playground is about studying the imports within the `tomodachi` application. It covers the 4 fundamental approaches:

- Deployment: we can deploy certain package and use it everywhere
- sys.path: we can append our module to the sys path and import it
- pathlib & env: we can re-read or load another .env.* file to configure another environment of our application
- importlib.util: we can dynamically import the module and re-use its content (functions, classes, variables, etc)

### Using the deployed package:

In [1]:
# uses our deployed one
from tomodachi_core.tomodachi.services import PandasService

# Shows no error? As long as you installed the deployed package!

### Using `sys` and `os` strategy

So, today I wanna check out this strategy:


In [2]:
import sys
import os
# this is a hack to get the tomodachi_core module to be found
# in the parent directory
# sys.path.append(os.path.join(os.getcwd(), ".."))

### Checking out the directory

In order to see all paths, we can use the `sys.path` and print it to the console as follows:

In [3]:
# let us try see what sys does
from pprint import pprint
pprint(sys.path)

['c:\\Program Files\\Python313\\python313.zip',
 'c:\\Program Files\\Python313\\DLLs',
 'c:\\Program Files\\Python313\\Lib',
 'c:\\Program Files\\Python313',
 '',
 'C:\\Users\\Lenovo\\AppData\\Roaming\\Python\\Python313\\site-packages',
 'C:\\Users\\Lenovo\\AppData\\Roaming\\Python\\Python313\\site-packages\\win32',
 'C:\\Users\\Lenovo\\AppData\\Roaming\\Python\\Python313\\site-packages\\win32\\lib',
 'C:\\Users\\Lenovo\\AppData\\Roaming\\Python\\Python313\\site-packages\\Pythonwin',
 'c:\\Program Files\\Python313\\Lib\\site-packages']


Let us use the magic line (append to the path)

We try to import, say, `test_file.py` which contains `greetings = 'Hello, world!'

Let us get it? So far, we know two approaches:

- We can use `pathlib` and then use absolute path to the thing.
- We can then use `importlib.util` to dynamically import the module & fetch its content

And now we try: sys for that.

In [4]:
# append the path
# Comment out the line below to see what happens and uncomment it (to make it run successfully)
# NB! After you comment it out, you need to restart the kernel for it to take effect
# sys.path.append(os.path.join(os.getcwd(), ".."))

# try to import 
from test_file import greetings
pprint(greetings) # it is a string 

ModuleNotFoundError: No module named 'test_file'

Result? Surprisingly, we got the correct output and we could import the module. 

Try to rerun the snippet above when commenting out the sys.path.append line? Result? You would see error. It will fail.

To provide my teammates with opportunity to run the code, I will use the following logic:

- We work around the pathlib.Path(__file__) by using `os.getcwd` instead
- We append to the path 
- Usually, we can already import, but in our case: the file does not exist, as we do not leave junk files
- Therefore, we use `with open()` to create a file and write to this (using the found path)
- After, we know that the file is guaranteed to exist as long as we specify the correct path, so we import
- Next up, we print the variable to the console
- Finally, we clean up resoucres by removing the path from sys and deleting the file.

You can comment out the deletion if you want to see what happens and check the file.

In [5]:
# Try approach 1 solution
import pathlib

# get the path to the root directory
get_current_dir = os.getcwd()
root_dir = pathlib.Path(get_current_dir).parents[1].resolve()

# but you can fix it by adding it to sys.path
sys.path.append(str(root_dir))

# we know that our file is located in the root directory
# so we can use the path to the file to get the path to the root directory
# if the file does not exist, create it and add the following to this file:
# greetings: str = "Hello, World!"

# create the file if it does not exist
file_path = root_dir / "test_file.py"

# write the content to this file
if not file_path.exists():
    with open(file_path, "w") as f:
        f.write('greetings: str = "Hello, World!"\n')

# try to import the file again
from test_file import greetings # Py-lint: shows that file does not exist, as clean-up removes it
pprint(greetings)

# clean up
# remove the path from sys.path
sys.path.remove(str(root_dir))

# remove the file
if file_path.exists():
    os.remove(file_path)


'Hello, World!'


The second approach would be using `importlib.util`, as shown in the `common/load_python_module.py`. You can run the code and see what happens there.

Basically, we again use the `pathlib` to get the path to root and then we go from path to the module path. After,
we can use this knowledge to import the module.

- We have to know: a) root path; b) path to the file
- We check if it exists (or we are certain it exists, we simply continue)
- We load the module 
- We use the module.

In [ ]:
import importlib.util

# we already know the current and root directories 
# from the previous example
# get the path to the file

file_path = root_dir / "tomodachi_core" / "config_development" / "config.py"

# check if the file exists
if file_path.exists():
    # load the module
    spec = importlib.util.spec_from_file_location("config", str(file_path))
    config = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(config)

    # use the module
    print(config.CSV_PATH) # it should print the path to the CSV file
else:
    print(f"File {file_path} does not exist")

Path C:\Users\Lenovo\Desktop\python_app\tuuleenergia_tomodachi exists.
./shared/data/raw/Synthetic_Wind_Power.csv
1 + 2 = 3


### Re-reading .env or creating new config

I won't do this here. I do not wanna do so here. I cannot as .env is the secret. However, I can and will explain.

First, you have to take a look at './common/shared_env_setup.py'. You can imagine that in addition to one .env, we 
can have another one - dedicated to the specific part of the codebase. In that case, that part of the codebase can
have its own config and load environmental variables from that file. 

In [ ]:
# NB! It will work if you add current directory to sys.path
# but it is not a good practice to do so
# so use vim to run the code below:
# python ./common/shared_env_setup.py
# Make sure that you are in "playground directory": cd ~/playground

# Or run:
# python ./common/manual_config_import.py

# get the path from the example
from common.shared_env_setup import CSV_PATH

# print the path
pprint(CSV_PATH) # it should print the path to the CSV file

ModuleNotFoundError: No module named 'playground'

### Deployed package usage

This is final way of using imports. In other words, we tell python that our package is installed, you gotta find it and import it for me.

How it works: 

- You create `pyproject.toml`
- You define your project: name, author, version, what to include, what to exlude, dependencies (in our case, we use requirements.txt)
- You build the project and if there are no errors, you call `python -m twince upload --repository testpypi dist/*`.
- Then you use your API_TOKEN to authorize and push to your cloud storate (TestPyPi)
- Once done, you can reuse your package within your application and add it to the depedency. 

In [ ]:
from tomodachi_core.tomodachi.utils.tools import check_timestamp

# check the timestamp
try:
    # mock data
    import pandas as pd
    data = {
        "Timestamp": ["2023-01-01 00:00:00", "2023-01-02 00:00:00", "2023-01-03 00:00:00", "2023-01-04 00:00:00", "2023-01-05 00:00:00"],
        "value": [10, 20, 30, 40, 50]
    }

    df = pd.DataFrame(data)
    
    # call the function 
    is_valid, _ = check_timestamp(df) # we do not specify column as long as it macthes the default one

    # print the result
    print(f"Is the timestamp valid? {is_valid}")
except Exception as e:
    print(f"Invalid timestamp: {e}")
finally:
    print("Done checking timestamp")
